In [27]:
# --- Imports & setup ---
import os, sys, warnings, math
import numpy as np
import pandas as pd
from pathlib import Path

# plotting (kept same style so your figs look identical)
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print('Python:', sys.version)


Python: 3.9.6 (default, Nov 11 2024, 03:15:38) 
[Clang 16.0.0 (clang-1600.0.26.6)]


In [28]:
# ==== Paths ====
from pathlib import Path
import pandas as pd
import numpy as np

DATA_DIR = Path('../data')
RAW_CSV  = DATA_DIR / 'ncr_ride_bookings.csv'
OUT_PKL  = DATA_DIR / 'churn_dataset.pkl'
OUT_CSV  = DATA_DIR / 'churn_dataset.csv'
OUT_PQ   = DATA_DIR / 'churn_dataset.parquet'

assert RAW_CSV.exists(), f"Can't find {RAW_CSV.resolve()}"

# ==== Quick sniff for column names ====
df_head = pd.read_csv(RAW_CSV, nrows=5, low_memory=False)
cols = df_head.columns.tolist()
print(f"Loaded file: {RAW_CSV}")
print("Detected columns (first 20):", cols[:20])

def pick(*cands):
    for c in cands:
        if c in cols:
            return c
    return None

COL_DATE   = pick('Date', 'Ride Date', 'Booking Date')
COL_TIME   = pick('Time', 'Ride Time', 'Booking Time')
COL_USER   = pick('Customer ID', 'User ID', 'Customer', 'CustomerID')
COL_STATUS = pick('Booking Status', 'Ride Status', 'Status')
COL_FARE   = pick('Booking Value', 'Fare', 'Amount', 'Price')
COL_DIST   = pick('Ride Distance', 'Distance (km)', 'Distance')
COL_VEH    = pick('Vehicle Type', 'Car Type', 'Vehicle')
COL_PAY    = pick('Payment Method', 'Payment', 'Payment Type')
COL_CR     = pick('Customer Rating', 'Customer Ratings', 'Cust Rating')
COL_DR     = pick('Driver Ratings', 'Driver Rating')

print("\nColumn mapping I will use:")
print("  USER   ->", COL_USER)
print("  DATE   ->", COL_DATE)
print("  TIME   ->", COL_TIME)
print("  STATUS ->", COL_STATUS)
print("  FARE   ->", COL_FARE)
print("  DIST   ->", COL_DIST)
print("  VEH    ->", COL_VEH)
print("  PAY    ->", COL_PAY)
print("  CR     ->", COL_CR)
print("  DR     ->", COL_DR)

# ==== Load full CSV ====
df = pd.read_csv(RAW_CSV, low_memory=False)

# ---- Build timestamp from Date + Time ----
if COL_DATE is not None and COL_TIME is not None:
    df['ts'] = pd.to_datetime(
        df[COL_DATE].astype(str).str.strip() + ' ' + df[COL_TIME].astype(str).str.strip(),
        errors='coerce'
    )
elif COL_DATE is not None:
    df['ts'] = pd.to_datetime(df[COL_DATE], errors='coerce')
elif COL_TIME is not None:
    df['ts'] = pd.to_datetime(df[COL_TIME], errors='coerce')
else:
    raise ValueError("Could not locate a Date/Time column to build timestamps.")

# ---- Basic cleaning ----
if COL_USER is None:
    raise ValueError("Could not locate a Customer/User ID column.")
df = df[df[COL_USER].notna() & df['ts'].notna()].copy()
df = df.sort_values([COL_USER, 'ts'])

# ---- Status normalization; count only COMPLETED as activity ----
if COL_STATUS is not None:
    st_norm = df[COL_STATUS].astype(str).str.lower().str.strip()
    print("\nTop Booking Status values:")
    print(st_norm.value_counts().head(10))
    COMPLETED = {
        'completed','ride completed','complete','finished','trip completed',
        'success','successful','ended','trip ended','completed ride'
    }
    df['__completed'] = st_norm.isin(COMPLETED)
else:
    df['__completed'] = True

df_act = df[df['__completed']].copy()

# ==== Per-user labeling at the PENULTIMATE ride ========================================
LOOKBACK_D = 90     # features from 90 days before the per-user cutoff
HORIZON_D  = 60     # label churn if NO completed ride within 60 days after cutoff
MIN_TOTAL_COMPLETED = 2   # user must have at least 2 completed rides overall (so a penultimate exists)
MIN_LOOKBACK_COMPLETED = 1

rows = []
user_groups = df_act.groupby(COL_USER, sort=False)

for uid, g in user_groups:
    g = g.sort_values('ts')
    if len(g) < MIN_TOTAL_COMPLETED:
        continue

    # --- per-user cutoff = time of the penultimate completed ride ---
    # (If exactly 2 rides, this is the first ride; if >2, it's the second-to-last.)
    ref_end = g['ts'].iloc[-2]

    # --- lookback window ends at ref_end; start LOOKBACK_D days before ---
    lb_start = ref_end - pd.Timedelta(days=LOOKBACK_D)
    in_win = g[(g['ts'] > lb_start) & (g['ts'] <= ref_end)].copy()
    if len(in_win) < MIN_LOOKBACK_COMPLETED:
        continue

    # --- future window for label: (ref_end, ref_end + HORIZON_D] ---
    future = g[(g['ts'] > ref_end) & (g['ts'] <= ref_end + pd.Timedelta(days=HORIZON_D))]

    # ---- features from lookback ----
    recency_days = (ref_end - in_win['ts'].max()).total_seconds() / (3600*24)
    rides_lb = len(in_win)

    row = {
        'Customer ID': uid,
        'num__recency_days': recency_days,
        'num__rides_lookback': rides_lb,
        'ref_end': ref_end
    }

    if COL_FARE in in_win.columns:
        fares = pd.to_numeric(in_win[COL_FARE], errors='coerce')
        row['num__fare_sum_lb']  = fares.sum()
        row['num__fare_mean_lb'] = fares.mean()
    if COL_DIST in in_win.columns:
        dd = pd.to_numeric(in_win[COL_DIST], errors='coerce')
        row['num__dist_sum_lb']  = dd.sum()
        row['num__dist_mean_lb'] = dd.mean()
    if COL_CR in in_win.columns:
        row['num__customer_rating_mean_lb'] = pd.to_numeric(in_win[COL_CR], errors='coerce').mean()
    if COL_DR in in_win.columns:
        row['num__driver_rating_mean_lb'] = pd.to_numeric(in_win[COL_DR], errors='coerce').mean()

    # time-of-day mix
    hr = in_win['ts'].dt.hour
    row['num__pct_morning'] = ((hr>=5)&(hr<12)).mean()
    row['num__pct_evening'] = ((hr>=17)&(hr<22)).mean()
    row['num__pct_night']   = ((hr>=22)|(hr<5)).mean()

    # most-used vehicle
    if COL_VEH in in_win.columns and in_win[COL_VEH].notna().any():
        row['cat__most_used_vehicle_lb'] = in_win[COL_VEH].value_counts().idxmax()

    # payment diversity
    if COL_PAY in in_win.columns:
        row['num__payment_diversity_lb'] = in_win[COL_PAY].nunique(dropna=True) / len(in_win)

    # ---- label ----
    row['churn_30d'] = int(len(future) == 0)  # 1 = churn (no completed ride within HORIZON_D)
    rows.append(row)

final = pd.DataFrame(rows)

# tidy & save
num_cols = [c for c in final.columns if c.startswith('num__')]
cat_cols = [c for c in final.columns if c.startswith('cat__')]
if 'ref_end' in final.columns:
    final.drop(columns=['ref_end'], inplace=True, errors='ignore')

OUT_PKL.parent.mkdir(parents=True, exist_ok=True)
final.to_pickle(OUT_PKL)
final.to_csv(OUT_CSV, index=False)
try:
    final.to_parquet(OUT_PQ, index=False)
except Exception:
    pass

# summary
print("\nSaving Dataset:")
print("="*50)
print(f"Dataset saved to {OUT_PKL}")
print(f"Dataset also saved to {OUT_CSV}")

print("\nFinal Dataset Summary:")
print("Shape:", final.shape)
print("Features:", len(num_cols) + len(cat_cols))
print("Churn rate:", round(final['churn_30d'].mean(), 3) if len(final) else float('nan'))
print("Customers:", len(final))


Loaded file: ../data/ncr_ride_bookings.csv
Detected columns (first 20): ['Date', 'Time', 'Booking ID', 'Booking Status', 'Customer ID', 'Vehicle Type', 'Pickup Location', 'Drop Location', 'Avg VTAT', 'Avg CTAT', 'Cancelled Rides by Customer', 'Reason for cancelling by Customer', 'Cancelled Rides by Driver', 'Driver Cancellation Reason', 'Incomplete Rides', 'Incomplete Rides Reason', 'Booking Value', 'Ride Distance', 'Driver Ratings', 'Customer Rating']

Column mapping I will use:
  USER   -> Customer ID
  DATE   -> Date
  TIME   -> Time
  STATUS -> Booking Status
  FARE   -> Booking Value
  DIST   -> Ride Distance
  VEH    -> Vehicle Type
  PAY    -> Payment Method
  CR     -> Customer Rating
  DR     -> Driver Ratings

Top Booking Status values:
Booking Status
completed                93000
cancelled by driver      27000
no driver found          10500
cancelled by customer    10500
incomplete                9000
Name: count, dtype: int64

Saving Dataset:
Dataset saved to ../data/churn